In [1]:
import tensorflow as tf
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

from random import randint

from load_data import gen_data,load_data,standardize

In [2]:
tf.reset_default_graph()

num_steps = 70


input_num = 5
output_num = 1

state_size = 100

num_layers = 20

def lrelu(x, name, leak=0.2): 
  #  return tf.maximum(x, leak * x, name=name) 
    return tf.maximum(x, 0, name=name) 

In [3]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, num_steps,input_num], name='input_placeholder')
y = tf.placeholder(tf.float32, [None], name='labels_placeholder')

keep_prob = tf.placeholder_with_default(1.0, shape=())



x_reshaped = tf.reshape(x,[tf.shape(x)[0],x.shape[1],1,x.shape[2]])



#conv1 = lrelu(tf.layers.conv2d(x_reshaped,name="conv1", filters = 8, 
#                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv1_lrelu")

#conv2 = lrelu(tf.layers.conv2d(conv1, name="conv2", filters = 16, 
#                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv2_lrelu")

#pool1 = tf.layers.max_pooling2d(inputs=conv2, name = "pool1", pool_size=[2, 1], strides=[2,1])

#conv3 = lrelu(tf.layers.conv2d(pool1, name="conv3", filters = 8, 
#                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv3_lrelu")
#bn3 = tf.contrib.layers.batch_norm(conv3,is_training=is_training, scope='bn3')

#conv4 = lrelu(tf.layers.conv2d(conv3, name="conv4", filters = 16, 
#                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv4_lrelu")
#bn4 = tf.contrib.layers.batch_norm(conv4,is_training=is_training, scope='bn4')


#pool2 = tf.layers.max_pooling2d(inputs=bn4, name = "pool2", pool_size=[2, 1], strides=[1,1])
rnn_inputs = tf.squeeze(x_reshaped,axis=2)


In [4]:
print(x_reshaped)
#print(conv1)
#print(conv2)
#print(pool1)


Tensor("Reshape:0", shape=(?, 70, 1, 5), dtype=float32)


In [5]:



cells = []
for i in range(0, num_layers):
    cell = tf.contrib.rnn.LSTMCell(state_size )
    #cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    cells.append(cell)
    
mul_cell = tf.nn.rnn_cell.MultiRNNCell(cells)

rnn_outputs, final_state = tf.nn.dynamic_rnn(mul_cell, rnn_inputs, dtype=tf.float32)



In [6]:
rnn_outputs_flat =  tf.reshape(rnn_outputs,[tf.shape(rnn_outputs)[0],
                                               rnn_outputs.shape[1]*rnn_outputs.shape[2]])
#dropout = tf.nn.dropout(dense1, keep_prob=keep_prob)

dense1 = tf.layers.dense(rnn_outputs_flat,1024,name="dense1")
dense2 = tf.layers.dense(dense1,256,name="dense2")
dense3 = tf.layers.dense(dense2,output_num,name="dense3")

logits = tf.squeeze(dense3,axis=1)

In [7]:
print(rnn_outputs)


Tensor("rnn/transpose:0", shape=(?, 70, 100), dtype=float32)


In [8]:


loss = tf.reduce_mean(tf.square(y - logits))
loss_sum = tf.summary.scalar('Loss', loss)

train_step = tf.train.AdamOptimizer(learning_rate = 0.00005).minimize(loss)

In [9]:
n_epochs = 150
batch_size = 32

merged = tf.summary.merge_all()
saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess = tf.Session()


resample="2H"

restore_checkpoint = False
save = False
tensorboard = True


if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
    saver.restore(sess, checkpoint_path)
else:
    sess.run(init)

In [ ]:
!rm -r logs/train

In [ ]:

    
checkpoint_path = "./logs/ET"
    
train_writer = tf.summary.FileWriter('logs/train',
                                    sess.graph)
step = 0
for e in range(n_epochs):
    for i,(x_,y_) in enumerate(gen_data(["Calgary","London"],['2013','2014'],
                                        batch_size,num_steps,resample=resample)):
        if(y_.shape[0] == 0):
            continue
            
        _,loss_,pred,summary = sess.run([train_step,loss,logits,merged],
                                        feed_dict = {x : x_, y: y_})
        
        if tensorboard:
            train_writer.add_summary(summary, step)
                
        if(e%1 == 0 and i % 25 == 0):
            print("Epoch {}  Iteration : {}  loss : {}  \nreal : {}   \npred : {}\n".format(e+1,i, loss_,
                                                                                           y_[0:3],pred[0:3]))
        step+=1
    if save:
        save_path = saver.save(sess, checkpoint_path)

Calgary Energy Balance 5min 2013
Calgary Water Balance 5min 2013
Calgary Lysimeters kg 5min2013


/notebooks/Recurrent/load_data.py:76: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  'Wind': "mean",'Rain': "sum",'Lysimeter': "mean"})


Epoch 1  Iteration : 0  loss : 233.9283447265625  
real : [ 14.22452777  14.2967424   14.27824058]   
pred : [ -3.23866118e-07  -3.60390516e-07  -4.19987714e-07]

Epoch 1  Iteration : 25  loss : 46.44373321533203  
real : [ 13.95170419  13.98073713  14.0080946 ]   
pred : [ 7.04055691  7.04087925  7.04131556]

Epoch 1  Iteration : 50  loss : 0.043749697506427765  
real : [ 13.81462143  13.8105867   13.81040362]   
pred : [ 13.55867863  13.55728054  13.55590916]

London Energy Balance 5min 2013
London Water Balance 5min 2013
London Lysimeters kg 5min2013
Epoch 1  Iteration : 75  loss : 6.573545932769775  
real : [ 20.44004445  20.29613432  20.12613734]   
pred : [ 21.17260361  21.17644691  21.18025208]

Epoch 1  Iteration : 100  loss : 3.201939582824707  
real : [ 13.74832375  13.74909517  13.74983534]   
pred : [ 15.49570751  15.49640465  15.4964323 ]

Epoch 1  Iteration : 125  loss : 0.08935105800628662  
real : [ 14.46308261  14.46743977  14.46327762]   
pred : [ 14.58888531  14.5885

In [11]:
for i,(x_,y_) in enumerate(gen_data(["Halifax"],['2013'],
                                    batch_size,num_steps,keep_data_loaded=False,resample=resample)):
    if(x_.shape[0] == 0):
        continue
        
    pred = sess.run([logits], feed_dict = {x : x_ })
    
    print("Iteration : {}  \nreal : {}   \npred : {}\n".format(i, y_[0:3],pred[0][0:3]))
    if(i > 200):
        break

Halifax Energy Balance 5min 2013
Halifax Water Balance 5min 2013
Halifax Lysimeters kg 5min2013


/notebooks/Recurrent/load_data.py:76: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  'Wind': "mean",'Rain': "sum",'Lysimeter': "mean"})


Iteration : 0  
real : [ 13.9695395   13.9451703   13.71566976]   
pred : [ 14.89901161  14.90154934  14.90408897]

Iteration : 1  
real : [ 13.43181661  13.42301212  13.43414838]   
pred : [ 14.90423298  14.90354919  14.90285587]

Iteration : 2  
real : [ 13.41358963  13.46298568  13.4986758 ]   
pred : [ 14.896698    14.896698    14.89782333]

Iteration : 3  
real : [ 13.82508431  13.70772717  13.68722838]   
pred : [ 14.90342903  14.90387726  14.90406418]

Iteration : 4  
real : [ 13.63754192  13.65461243  13.67027694]   
pred : [ 14.90041924  14.89808369  14.89963436]

Iteration : 5  
real : [ 13.7131516   13.64565815  13.64586114]   
pred : [ 14.90329552  14.89986229  14.89950943]

Iteration : 6  
real : [ 13.9216939   13.90814628  13.88271784]   
pred : [ 14.8992691   14.89993095  14.899436  ]

Iteration : 7  
real : [ 13.95961154  13.97929617  13.94904479]   
pred : [ 14.89915657  14.89962387  14.90008354]

Iteration : 8  
real : [ 13.99191449  14.03833693  14.03416633]   
pred 

Iteration : 72  
real : [ 16.0890208   16.07037986  16.02989003]   
pred : [ 14.9015398   14.90293884  14.90347481]

Iteration : 73  
real : [ 15.92438596  15.92074116  15.91706969]   
pred : [ 14.90217018  14.90138626  14.90116692]

Iteration : 74  
real : [ 15.9060086   15.9056391   15.91639968]   
pred : [ 14.90298462  14.90311432  14.90290546]

Iteration : 75  
real : [ 15.87448198  15.93815952  16.03497835]   
pred : [ 14.9030304   14.90130138  14.90109539]

Iteration : 76  
real : [ 16.00943825  16.00182143  15.99472368]   
pred : [ 14.90233517  14.90336227  14.90152931]

Iteration : 77  
real : [ 15.95147575  15.90779505  15.85482212]   
pred : [ 14.90153217  14.9012146   14.90058517]

Iteration : 78  
real : [ 15.78955185  15.78361801  15.78211489]   
pred : [ 14.90046597  14.9006691   14.90018654]

Iteration : 79  
real : [ 15.77448246  15.76859423  15.75822803]   
pred : [ 14.89730263  14.90019703  14.90240288]

Iteration : 80  
real : [ 15.65140866  15.63976153  15.63742662]

Iteration : 144  
real : [ 15.90016448  15.8892201   15.87688848]   
pred : [ 14.90311241  14.90307713  14.90258694]

Iteration : 145  
real : [ 15.84947983  15.84852514  15.84711978]   
pred : [ 14.90219879  14.90200806  14.90209675]

Iteration : 146  
real : [ 15.96686627  15.96209123  16.12736426]   
pred : [ 14.89830875  14.90007687  14.90148354]

Iteration : 147  
real : [ 16.398197    16.36146419  16.35063019]   
pred : [ 14.90213013  14.90291214  14.90343761]

Iteration : 148  
real : [ 16.34151976  16.33686014  16.33379467]   
pred : [ 14.89879894  14.89723015  14.89723682]

Iteration : 149  
real : [ 16.31155946  16.27835536  16.24006766]   
pred : [ 14.90325642  14.90361118  14.90362549]

Iteration : 150  
real : [ 16.20529645  16.20595256  16.20858385]   
pred : [ 14.9042263   14.90292072  14.89987373]

Iteration : 151  
real : [ 16.21359865  16.20914174  16.18499892]   
pred : [ 14.89908028  14.90233421  14.90379715]

Iteration : 152  
real : [ 16.07967602  16.07541179  16.

In [15]:
data = next(load_data(["Calgary"],['2013']))

In [17]:
print(data)

                      Radiation      Wind   Humidity  Lysimeter       Temp  \
2013-05-16 12:00:00   16.256182  1.259000  63.416364  12.874815  10.451818   
2013-05-16 18:00:00  -49.123806  0.820819  80.275000  12.998047   7.874569   
2013-05-17 00:00:00  -62.440694  1.786375  90.811111  13.022708   3.558889   
2013-05-17 06:00:00  176.045694  1.313750  72.180972  13.000165   7.504472   
2013-05-17 12:00:00  324.389722  0.763486  36.864861  15.199888  15.742361   
2013-05-17 18:00:00  -11.024542  0.487528  37.742222  15.209717  14.858194   
2013-05-18 00:00:00  -52.335139  1.245139  69.095417  15.183530   6.657917   
2013-05-18 06:00:00  243.985583  1.778542  57.481944  15.141670  10.006958   
2013-05-18 12:00:00  397.068056  2.130014  33.206667  14.951719  17.845833   
2013-05-18 18:00:00  -20.653514  1.849778  44.300556  14.832475  14.691250   
2013-05-19 00:00:00  -25.777000  1.316125  67.887222  14.810025   9.433556   
2013-05-19 06:00:00   30.498292  1.157722  66.066250  14.860086 